# Partie 1


## Task 1

In [ ]:
!pip install datasets sklearn-crfsuite transformers beautifulsoup4 nltk spacy rdflib seqeval pykeen

In [ ]:
import string
import nltk
nltk.download('stopwords')

In [ ]:
from datasets import load_dataset
# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003")
# Access the training, validation, and test sets
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']




In [ ]:
import string

def remove_punctuation(input_list):
    """Removes tokens that consist entirely of punctuation and returns the cleaned list with removed indexes."""
    cleaned_list = []
    removed_indexes = []

    for idx, token in enumerate(input_list):
        if all(char in string.punctuation for char in token):
            removed_indexes.append(idx)  # On stocke l'index du token supprimé
        else:
            cleaned_list.append(token)  # On garde le token valide

    return cleaned_list, removed_indexes


input_str = ['CRICKET', '-', 'LEICESTERSHIRE', 'TAKE', 'OVER', 'AT', 'TOP', 'AFTER', 'INNINGS', 'VICTORY', '.']

# Call the function to remove punctuation tokens
cleaned_str,index = remove_punctuation(input_str)

# Print the cleaned string
print(cleaned_str)
print(index)

In [ ]:
# Fonction pour retirer les stopwords
from nltk.corpus import stopwords

def remove_stopwords(tokens):
    """Removes stopwords from the token list and returns the cleaned list with removed indexes."""
    stop_words = set(stopwords.words("english"))
    stop_words.update(["n't", "'s"])  # Ajout de contractions spécifiques

    cleaned_tokens = []
    removed_indexes = []

    for idx, word in enumerate(tokens):
        if word.lower() in stop_words:
            removed_indexes.append(idx)  # Stocke l'index du mot supprimé
        else:
            cleaned_tokens.append(word)  # Conserve les tokens valides

    return cleaned_tokens, removed_indexes


# Liste de mots déjà tokenisés
text = ['"', 'We', 'do', "n't", 'support', 'any', 'such', 'recommendation', 'because', 'we', 'do', "n't", 'see', 'any', 'grounds', 'for', 'it', ',', '"',
        'the', 'Commission', "'s", 'chief', 'spokesman', 'Nikolaus', 'van', 'der', 'Pas', 'told', 'a', 'news', 'briefing', '.']

# Appel de la fonction
filtered_text,index = remove_stopwords(text)
print(filtered_text)
print(index)


In [ ]:
def text_lowercase(texts):
    return [text.lower() for text in texts]

# Appel de la fonction
input_str = ['CRICKET', '-', 'LEICESTERSHIRE', 'TAKE', 'OVER', 'AT', 'TOP', 'AFTER', 'INNINGS', 'VICTORY', '.']
filtered_text=text_lowercase(input_str)
print(filtered_text)

In [ ]:
def remove_indices(lst, indices):
    """Supprime les éléments d'une liste aux index spécifiés."""
    indices_set = set(indices)  # Utiliser un set pour optimisation
    return [item for idx, item in enumerate(lst) if idx not in indices_set]
lst = ["a", "b", "c", "d", "e"]
indices = [1, 3]
filtered_lst = remove_indices(lst, indices)
print(filtered_lst)  # Sortie : ["a", "c", "d", "e



In [ ]:
def clean_tokens_and_tags(tokens, ner_tags):
    """ Nettoie les tokens et garde uniquement les ner_tags des tokens restants """
    cleaned_token = text_lowercase(tokens)  # Minuscule
    cleaned_token,index_1 = remove_punctuation(cleaned_token)
    cleaned_token, index_2 = remove_stopwords(cleaned_token)

    ner_tags = remove_indices(ner_tags, index_1)
    ner_tags = remove_indices(ner_tags, index_2)

    # Convert ner_tags elements to strings
    ner_tags = [str(tag) for tag in ner_tags]

    return cleaned_token, ner_tags




In [ ]:
X_train = []
y_train = []

X_test = []
y_test = []

#Apply this function to each elements
for i in range(len(train_dataset)):
    token,tag= clean_tokens_and_tags(
        train_dataset[i]["tokens"], train_dataset[i]["ner_tags"]
    )
    X_train.append(token)
    y_train.append(tag)

for i in range(len(test_dataset)):
    token,tag = clean_tokens_and_tags(
        test_dataset[i]["tokens"], test_dataset[i]["ner_tags"]
    )
    X_test.append(token)
    y_test.append(tag)

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from datasets import load_dataset
import spacy
from spacy.training import Example
from spacy.scorer import Scorer
from seqeval.metrics import classification_report, f1_score

# Train the CRF model
crf = sklearn_crfsuite.CRF(
 algorithm='lbfgs',
 c1=0.1,
 c2=0.1,
 max_iterations=100,
 all_possible_transitions=False
)
# Fit the Crf model
crf.fit(X_train, y_train)

# Predict using Crf model
y_pred_crf = crf.predict(X_test)

# Evaluate the model
precision_crf = metrics.flat_precision_score(y_test, y_pred_crf, average='weighted')
recall_crf = metrics.flat_recall_score(y_test, y_pred_crf, average='weighted')
f1_crf = metrics.flat_f1_score(y_test, y_pred_crf, average='weighted')

print("         === Évaluation of CRF model ===")
print(metrics.flat_classification_report(y_test, y_pred_crf))

In [ ]:
!unzip /en_ner_conll03.zip

In [ ]:
# Load spaCy's pre-trained NER model
nlp = spacy.load("best_ner_model")
# Example text
text = "Apple was founded by Steve Jobs."
# Process the text with spaCy
doc = nlp(text)
# Extract named entities
entities = [(ent.text, ent.label_) for ent in doc.ents]
print(entities)


In [ ]:
# We fetch ner from our pipeline
ner = nlp.get_pipe("ner")
# Convert the data so we can train spacy model
train_data = []
for words, labels in zip(X_train, y_train):
    entities = []
    start = 0
    for word, label in zip(words, labels):
        end = start + len(word)
        if label != "O":
            entities.append((start, end, label))
        start = end + 1
    train_data.append((" ".join(words), {"entities": entities}))

# Add entities to the spacy model
for label in set(label for labels in y_train for label in labels if label != "O"):
    ner.add_label(label)

# Train the spacy model
optimizer = nlp.begin_training()
for epoch in range(3):  # 3 itérations
    losses = {}
    for text, annotations in train_data:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], losses=losses, drop=0.5, sgd=optimizer)
    print(f"Époque {epoch+1}, Pertes : {losses}")


In [ ]:
# We try our model on X_Test
y_pred_spacy = []
for words in X_test:
    doc = nlp(" ".join(words))
    pred_labels = ["O"] * len(words)  # Par défaut, O (pas d'entité)
    for ent in doc.ents:
        # Check if the entity's first token is in the words list
        first_token = ent.text.split()[0]
        if first_token in words:
            start_idx = words.index(first_token)  # Trouver le mot de départ
            pred_labels[start_idx] = ent.label_
    y_pred_spacy.append(pred_labels)


precision_spacy = metrics.flat_precision_score(y_test, y_pred_spacy, average='weighted')
recall_spacy = metrics.flat_recall_score(y_test, y_pred_spacy, average='weighted')
f1_spacy = metrics.flat_f1_score(y_test, y_pred_spacy, average='weighted')

print("\n=== Evaluation of Spacy model ===")
# Get unique labels from y_test
unique_labels = set([label for sublist in y_test for label in sublist])
# Remove "O" if present
unique_labels.discard("O")

# Print classification report with labels if found
if unique_labels:
    print(metrics.flat_classification_report(y_test, y_pred_spacy))
else:
    print("No entities found in the dataset for classification report.")

Let's compare both models using accuracy, F1 score and Recall

In [ ]:
# === Comparison Bewtween Both models ===
# The spacy model has been trained twice (the first time by the teacher and then by us above)

print("\n=== Comparaison of both models : Spacy vs Crf ===")
print(f"Modèle      | Précision  | Rappel     | F1-score")
print(f"CRF        | {precision_crf:.4f}   | {recall_crf:.4f}   | {f1_crf:.4f}")
print(f"spaCy NER   | {precision_spacy:.4f}   | {recall_spacy:.4f}   | {f1_spacy:.4f}")

In [ ]:
import spacy
# Load spaCy's pre-trained model
nlp = spacy.load("en_core_web_sm")
# Example text
text = "Apple was founded by Steve Jobs."
# Process the text with spaCy
doc = nlp(text)
# Extract relations
relations = []
for token in doc:
  if (token.dep_ == "nsubj" or token.dep_ == "nsubjpass") and token.head.dep_ == "ROOT":
    subject = token.text
    predicate = token.head.text
    for child in token.head.children:
      if child.dep_ == "prep" or child.dep_ == "agent":
        for obj in child.children:
          if obj.dep_ == "pobj":
            relations.append((subject, predicate, obj.text))
print(relations)


In [ ]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS
# Create a new RDF graph
g = Graph()
# Define namespaces
EX = Namespace("http://example.org/")
# Add triples to the graph
g.add((URIRef(EX.Apple), RDF.type, URIRef(EX.Company)))
g.add((URIRef(EX.SteveJobs), RDF.type, URIRef(EX.Person)))
g.add((URIRef(EX.Apple), URIRef(EX.founded_by), URIRef(EX.SteveJobs)))
# Serialize the graph in RDF/XML format
print(g.serialize(format="xml"))
# Perform a SPARQL query
query = """
SELECT ?subject ?predicate
WHERE {
 ?subject ?predicate <http://example.org/SteveJobs>
}
"""
for row in g.query(query):
 print(f"{row.subject} {row.predicate} ")

## Task 2

In [ ]:
! pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time
import random
import re
import undetected_chromedriver as uc

def fetch_articles():
    # Initialize the Chrome webdriver
    options = uc.ChromeOptions()

    driver = uc.Chrome(options=options)
    driver.get("https://techcrunch.com/latest/")

    sleep_time = random.uniform(5, 10)
    time.sleep(sleep_time)

    bouton1=driver.find_element(By.XPATH,"/html/body/div[1]/div/div/div/div/div/div[3]/button[3]")
    bouton1.click()

    sleep_time = random.uniform(2, 5)
    time.sleep(sleep_time)
    page_source = driver.page_source
    # Close the browser
    driver.quit()
    articles = []
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(page_source, "html.parser")

    for article in soup.find_all(class_="wp-block-techcrunch-card wp-block-null",limit=10):
        sleep_time = random.uniform(2, 5)
        time.sleep(sleep_time)
        info=article.find(class_='loop-card__title').find("a")
        try:
          title = info.get_text()
        except AttributeError:
          title = "no title"
        article_url = info['href']
        print(article_url)
        article_response = requests.get(article_url)
        article_soup = BeautifulSoup(article_response.content,'html.parser')
        sleep_time = random.uniform(2, 5)
        time.sleep(sleep_time)

        try:
          block = article_soup.find_all('p',class_="wp-block-paragraph")
          # Initialisation d'une variable pour stocker tout le contenu de l'article
          content = ""

          # Boucle sur chaque paragraphe et ajout du texte dans `content`
          for paragraph in block:
              content += paragraph.get_text() + "\n"
          articles.append({'title': title,'url': article_url,'content': content})
        except AttributeError:
          print("1 erreur")
    return articles

articles = fetch_articles()


for article in articles:
  print(article['title'])
  print(article['content'][:200]) # Print the first 200 characters of the content
  print()

In [ ]:
print(articles)

In [ ]:
articles = [{'title': 'The Cadillac Optiq EV and Cadillac Escalade IQ go big on tech, luxury, and price', 'url': 'https://techcrunch.com/2025/03/24/the-cadillac-optiq-ev-and-cadillac-escalade-iq-go-big-on-tech-luxury-and-price/', 'content': 'Cadillac’s new IQ portfolio of all-electric vehicles — notably the compact Optiq SUV and grandaddy full-size Escalade IQ — has put the company on a techier, more advanced, and luxurious plane. And the shift couldn’t have come soon enough.\xa0\nThe slate of traditional internal combustion engine vehicles that have come out of Cadillac in recent years hasn’t quite matched the levels of luxury and tech of its rivals.\xa0\nCadillac aims to change that with its debut of the compact Optiq SUV as well as the Escalade IQ, which take big swings on tech and advanced features. Importantly, Cadillac gave these two EVs distinct luxury personalities in an effort to appeal to different consumers. Call it the Young and Fun and the Old Money Luxe.\nCadillac let me loose in Northern California over two days to drive the Optiq SUV and Escalade IQ. Here’s how the two newly launched EVs stacked up.\xa0\nWhen I learned that engineers from my beloved Cadillac Blackwing had a hand in the Optiq crossover, I was immediately intrigued to see what 300 horsepower and 354 pound-feet of torque would feel like in this all-wheel drive crossover on the twisties.\xa0\nIt’s pretty darn entertaining. Yes, all EVs have smooth power delivery and the ability to accelerate quickly out of corners, but not every one has the suspension to back it up.\xa0\nThe dampers here get what Cadillac calls Passive Plus tuning. While cornering at high speeds the car remains flat and stable, but it can handle broken pavement without getting upset. Essentially, it can react to both slow and quick events to provide handling that is sporty and comfortable, without the complication of electronics. Of course, I’d welcome a future V-Performance Optiq with MagneRide dampers for even more giggles, but for now the hotrod kid in me loves this thing.\xa0\nIf you haven’t carved a canyon in an EV before, it can be a bit different than in a traditional vehicle, thanks to regenerative braking.\nThe Optiq gives you the ability to turn it off and just rely on the mechanical brakes or keep them at normal for just a bit of slow down when you lift off the throttle. I like maximum regen as it adds another level of skill to lift off the accelerator pedal at the right moment. There’s also a paddle behind the steering wheel that can add regen braking even if the system is set for none. This also can add another skill element for those looking to test their abilities. It’s the gamification of driving.\xa0\nMeanwhile, the drive experience in the three-row Escalade IQ is pure comfy goodness. Even though it sports the more advanced MagneRide dampers, those seem to mainly counteract the vehicle’s brutal 9,000-pound heft.\xa0\nThe three-row SUV’s full 700 horsepower and 785 pound-feet of torque are only accessible in a mode called Velocity Max. Otherwise the SUV makes due with 680 ponies and 615 pound-feet of torque going to all four wheels. That might sound like overkill, but this zaftig baby needs it. Anything less and it would feel severely underpowered.\nThe Escalade IQ is all about smooth, comfortable driving. U-turns are easy thanks to a rear-wheel steer system that makes the 136-inch wheelbase seem much shorter. If you want to arrive in style an Arrival mode lets the Escalade move diagonally into a parking spot. Just be wary of what’s around you as it’s easy to underestimate the angle.\xa0\nCadillac also stacked the Escadale with luxury and convenience features. Press on the brake pedal and the doors close automatically. The vehicle also comes with heated seats and a massage function.\xa0\nAnd then there is the advanced driver assistance system. Both vehicles get Cadillac’s SuperCruise standard that allows for hands-free/eyes-up driving on 750,000 miles of road.\xa0\nThe system can change lanes when the driver activates the turn signal or it will do it on its own in order to pass a slower moving car. It was raining and pretty foggy while SuperCruising in the Escalade and though the system hesitated while executing a lane and a sweeping turn at the same time, it had no problems reading lane markings through the mist.\xa0\nThere are plenty of standard safety features on both EVs as well. Blind spot monitoring, forward collision alert and braking, lane-keeping assist — all of those features are here.\nI especially like the rear camera mirror on both vehicles that gives a wide-angle view of what’s going on behind the car. However, the Escalade IQ goes a step further with standard turn-signal activated camera display. Sure, the blind-spot monitoring is enough, but the video display on the screen that shows the adjoining two lanes is pretty sweet.\nThe little Optiq has an 85 kWh battery with enough electrons for 302 miles of driving. The 400-volt system can accept a DC fast charge of 150 kW, quick enough to get about 79 miles of range in 10 minutes.\xa0\nThe Escalade IQ, however, goes gigantor with a 205 kWh battery with 460 miles of range. Like the related Hummer EV, the Escalade’s battery is actually two 400-volt battery packs that can work in series or parallel, enabling a DC fast charge of up to 350 kW and the ability to gain 100 miles of range in 10 minutes. Both are on the CCS charging platform, but a NACS adaptor can be purchased from Cadillac for $225.\xa0\nFor home charging, both EVs come standard with an 11.5 kW onboard charger, but a 19.2 kW system can be had as well. This should enable about 36 miles per hour when charging at home.\xa0\nThe interior of the Optiq definitely skews younger and hipper, with recycled materials, a clean look and some nifty storage solutions.\xa0\nThere is 33 inches of diagonal screen space and a seriously kicking 19-speaker AKG Audio system with Dolby Atmos. The Atmos system is pretty dope, with sounds bouncing around the cabin in a way that delighted even my deficient left ear, to say nothing of my fully-functional right ear.\xa0\nMeanwhile, the Escalade IQ appeals to an older, more sophisticated crowd, but still goes big on tech with 55 inches of screen stretching across the entire dash. The passenger screen can play movies or browse the web, all without disturbing the driver.\xa0\nThere is also a center screen below that’s used for HVAC controls, switching to automatic door controls when the SUV is in park. The two wireless charging pads for the front row are a nice touch. Unfortunately, the Dolby Atmos is not available here thanks to development timelines, but look for it in 2026.\nNeither vehicle has Apple CarPlay or Android Auto, instead relying on Google built-in for maps and smartphone interaction.\nWhile I like that Google maps suggests charging stations and estimates remaining range at the end of a journey, I had a hard time setting up Google Assistant to read my text messages. I never got it to work in the Optiq and I had to have an engineer walk me through it in the Escalade, and even then it took a few tries. Once connected my phone posted a notification when I received a text message and allowed a voice-to-text reply, but I didn’t have time to see if it could place a call or play a podcast.\xa0\nThe Optiq is the entry-level EV in Cadillac’s portfolio, starting at $55,615 including $1,495 for destination for the Luxury 1 trim. However, the Monarch Orange paint with the black roof is only available on the Sport 1 and Sport 2 trims and is well worth the extra dollars.\nThe Escalade IQ is a whole other ballgame. The Luxury 1 trim starts at nearly $130,000 including $2,290 for destination. All paint colors are available across all trims, but the Midnight Steel Frost for $5,925 is a stunner. The most you could drop for a starting price is just over $150,000 for the Sport 2 trim, which has niceties like ventilated second row seats, a 38-speaker AKG sound system, available second row massage and second row executive seat package as well as a thermal camera for night vision.\xa0\nMy time in the Optiq left a positive impression — even if I’m\xa0 likely a bit older than the intended demographic. The car’s on-road manners appeal to the thrill-seeker in me.\xa0\nYes, the Escalade is cool, but it’s more of a lifestyle and status marker, something that doesn’t get me revved up, if you’ll pardon the expression.\nRegardless, both offer up some of the best tech in the segment. Safety features are all standard as is SuperCruise and the myriad screens give both cabins a futuristic element. I’m not happy with the lack of Apple CarPlay and Android Auto, but that is one quibble with what are both outstanding vehicles.\nTopics\n© 2025 Yahoo.\n'}, {'title': '23andMe faces an uncertain future — so does your genetic data\xa0', 'url': 'https://techcrunch.com/2025/03/24/23andme-faces-an-uncertain-future-so-does-your-genetic-data/', 'content': 'DNA and genetic testing firm 23andMe is in turmoil following a 2023 data breach and its ongoing financial decline. The once-pioneering giant now faces an uncertain future as bankruptcy looms over the company, intensifying concerns about what might happen to the genetic data of 23andMe’s some 15 million customers.\nBest known for its saliva-based test kits that offer a glimpse into a person’s genetic ancestry, 23andMe has seen its value plummet more than 99% from its $6 billion peak since going public in early 2021 after failing to turn a profit.\nThat lack of profit was attributed to waning consumer interest in 23andMe’s use-once test kits and lackluster growth of its subscription services. The company was also floored by a huge months-long data breach that saw hackers steal the ancestry data of almost 7 million users throughout 2023. The company agreed in September to pay $30 million to settle a lawsuit related to the breach.\xa0\nLess than a week later, 23andMe founder and CEO Anne Wojcicki said she was “considering third-party takeover proposals” for the company. Wojcicki quickly walked back the statement, instead saying she planned to take the company private. But the damage was done, and all of the company’s independent board members resigned with immediate effect.\nAfter filing for bankruptcy protection in March 2024, the company’s assets — including vast banks of DNA data — will be sold through a court-supervised sale. Wojcicki also resigned from the company.\nWhere does that leave millions of people’s genetic data?\nAs evidenced by its 2023 data breach, which saw hackers steal information such as users’ genetic predisposition and ancestry reports, 23andMe collects a ton of information on its users.\nIf you’re one of the many millions that have shipped your saliva to 23andMe to learn about your ancestry, you may have assumed that this data will remain private under law, such as the Health Insurance Portability and Accountability Act. HIPAA, as it’s known, sets the standards for protecting sensitive health information from being disclosed without a person’s knowledge or consent.\xa0\nHowever, 23andMe is not a company covered under HIPAA. As such, 23andMe is largely bound only by its own privacy policies, which it can change at any time.\nAndy Kill, a spokesperson for 23andMe, told TechCrunch that the company believes this is a “more appropriate and transparent model for the data we handle, rather than the HIPAA model employed by the traditional healthcare industry.”\xa0\nA lack of federal regulation and a cluttered mess of state privacy laws ultimately means that if 23andMe faces a sale, the data of millions of Americans is also on the table. The company’s privacy policy says that its customers’ personal information “may be accessed, sold or transferred” as part of a bankruptcy, merger, acquisition, reorganization, or sale.\nThe fact that customer data is a saleable asset has also been made clear by Wojcicki, who reportedly told investors that 23andMe will no longer pursue its cost-intensive drug development programs and will instead focus on marketing its vast database of customer data to pharmaceutical companies and researchers.\n23andMe maintains that its data privacy policies would not change in the event of a sale. These policies state that the company will never share users’ information with insurance companies, or with law enforcement without a warrant. The latter have increasingly turned to third-party DNA companies for genetic information, but 23andMe has so far resisted all U.S. law enforcement requests for such data, according to its long-running transparency report.\xa0\nPotential buyers of 23andMe may have entirely different ideas about how to use the company’s potentially valuable trove of DNA data. Privacy advocates at the digital rights group Electronic Frontier Foundation have already urged 23andMe to resist a sale to any company with ties to law enforcement, warning that customers’ genetics data could be used by police to indiscriminately search for evidence of crimes.\n“Our own commitment to apply the terms of our privacy policy to the personal information of our customers in the event of a sale or transfer is clear: the 23andMe Terms of Service and Privacy Statement would remain in place unless and until customers are presented with, and agree to, new terms and statements — and only after receiving appropriate notice of any new terms, under applicable data protection laws,” Kill told TechCrunch.\xa0\nNow as 23andMe faces bankruptcy, there are calls for 23andMe customers to take action now to protect their data from being sold.\nCalifornia’s attorney general Rob Bonta said in a statement after 23andMe filed for bankruptcy that state residents have a right to demand the deletion of their genetic data under state law.\xa0\nMeredith Whittaker, the president of end-to-end encrypted messaging app Signal, said in a post on X: “It’s not just you. If anyone in your family gave their DNA to [23andMe], for all of your sakes, close your/their account now.”\nEva Galperin, the director of cybersecurity at the EFF, also warned users to take action. “If you have a 23andMe account, today is a good day to login and request the deletion of your data,” said Galperin in a post on X.\nRequesting the deletion of your data on 23andMe is relatively easy.\xa0\nLog in to your 23andMe account and navigate to Settings > Account Information > Delete Your Account. 23andMe will prompt you to confirm your decision, warning that deleting your account is permanent and irreversible.\nThere is an important caveat. As noted in 23andMe’s privacy policy, account deletion is “subject to retention requirements and certain exceptions,” which means the company may hold on to some of your data for an unspecified amount of time.\xa0\nFor example, 23andMe will retain your genetic information, date of birth, and gender “as required for compliance” and will retain limited data related to your deletion request, “including but not limited to, your email address, account deletion request identifier, communications related to inquiries or complaints and legal agreements.”\nSimilarly, if you’ve already agreed to 23andMe sharing your data for research purposes, you can reverse that consent, but there’s no way for you to delete that information. Kill tells TechCrunch that around 80% of 23andMe customers — roughly 12 million people — consent to participate in its research program.\xa0\nFirst published on October 19, 2024 and updated since.\nTopics\n© 2025 Yahoo.\n'}, {'title': 'AI chip startup FuriosaAI reportedly turns down $800M acquisition offer from Meta', 'url': 'https://techcrunch.com/2025/03/24/ai-chip-startup-furiosaai-reportedly-turns-down-800m-acquisition-offer-from-meta/', 'content': 'Posted:\nFuriosaAI, a South Korean startup that makes chips for AI applications, has rejected an $800 million acquisition offer from Meta, opting instead to focus on developing and producing its AI chips, according to a local media report.\nDisagreements over post-acquisition business strategy and organizational structure, rather than price issues, caused the negotiations to break down, the report said.\nAlong with other tech companies building large language models (LLMs) for various AI applications, Meta has been trying to reduce its reliance on Nvidia for chips that are specialized for training and building LLMs. The tech giant last year unveiled its custom AI chips, and in January said it would invest up to $65 billion this year to support its AI initiatives.\nFuriosaAI did not respond to a request for comment. Meta did not immediately respond to a request for comment outside regular business hours.\nMeanwhile, FuriosaAI is reportedly in talks with investors to raise approximately $48 million (KRW 70 billion), and aims to complete the fundraise this month.\nFounded in 2017 by June Paik, who previously worked at Samsung Electronics and AMD, FuriosaAI has developed two AI chips, called Warboy and Renegade (RNGD), to take on the likes of Nvidia and AMD. \nThe startup has said it has completed testing the RNGD chips, which are said to be best suited for reasoning models, in partnership with LG AI Research and Aramco. LG AI Research reportedly plans to\xa0use RNGD\xa0chips in its AI infrastructure, and the startup plans to launch the chips later this year.\n\nTopics\n© 2025 Yahoo.\n'}, {'title': 'Ex-Network International execs raise $6.75M for Enza, an African fintech serving banks', 'url': 'https://techcrunch.com/2025/03/24/ex-network-execs-raise-6m-for-enza/', 'content': 'Over the past decade, Dubai-based Network International has become one of the dominant payment processors across the Middle East and Africa, thanks in part to a pair of acquisitions.\nHowever, many large incumbents can fall prey to slower innovation, opening the door for smaller, faster-moving startups. The latest development is Enza, a fintech founded in 2022 by Hany Fekry, a former managing director at Network, along with another ex-Network executive Hamish Houston.\nThe fintech, which has raised $6 million in seed funding, is building infrastructure for banks and fintechs, offering a range of local payment solutions, from cards to wallets to real-time payments.\xa0\nBefore launching Enza, the founders managed global acceptance, processing, and consumer finance departments at Network International. While Network was building a robust payments network across the Middle East and Africa, focusing primarily on the acceptance side of things, they felt a massive gap in creating comprehensive solutions for banks and fintechs, especially in Africa.\nWhen neither party could find an alignment with Network, they resigned to start Enza, which officially launched in January 2023.\xa0 \n“Our divergence prompted us to take a step back and rethink how to address these underserved needs in the market,” CEO Fekry told TechCrunch.\xa0\nThe founders of Enza say they’ve built the company using lessons from their time at Network International and its subsidiary, DPO Group. But unlike those firms, which focused largely on card acceptance and merchant acquiring, Enza is taking a broader approach, serving both sides of the transaction.\nEnza’s platform is designed for banks and fintechs on the issuing side, and SMEs and merchants on the acceptance side. The startup is initially targeting Egypt, Nigeria, and South Africa, three of the continent’s largest financial markets.\nPayments are often the first entry point into formal finance for the millions of underserved or unbanked small businesses across Africa. Enza wants to help these businesses accept in-person and online payments at little to no cost — a strategy it thinks will allow banks and fintechs to build long-term relationships.\nOnce those are in place, Enza’s infrastructure enables cross-selling of lending, savings, insurance, and other financial services.\n“Payments are the gateway,” says Andrew Key, who joined Enza as an executive director last year. “But the value is in the data and the services you can layer on top.”\nThat strategy also plays to the changing dynamics between banks and fintechs in Africa. For years, banks have ceded infrastructure and particularly SME market share to players like Flutterwave, Fawry, Paymob, and Moniepoint, now Nigeria’s largest merchant acquirer. But banks still hold key advantages, namely broader service offerings and regulatory backing.\n“Banks have realized they gave up too much ground to fintechs,” Houston said. “We want to give them the tech to compete and win it back.”\nSimilarly, despite the rise of fintechs across Africa, banks remain the central, regulated players behind most payment aggregators. But many still lack clear visibility into what their aggregator partners or downstream merchants are doing.\xa0\nThat’s one of Enza’s functionalities, the founders say: Giving banks more transparency and control over their payment ecosystems so they can stay compliant while scaling.\nThe Abu Dhabi-based startup also broadens the payment options available to banks. Enza integrates with local card schemes like Verve, AfriGo, and Meeza, alongside global networks like Visa and Mastercard.\xa0\nIt also connects with real-time payment infrastructure, including Nigeria’s NIBSS, South Africa’s PayShap, and Egypt’s InstaPay, as well as mobile money and telco wallets, while supporting QR codes, buy-now-pay-later (BNPL), and contactless payments features.\nEnza is leveraging its founders’ decades of experience and deep relationships across the continent to quickly secure contracts with several banks. For instance, Fekry previously served as chief commercial officer at Emerging Markets Payments (EMP), which was acquired by Network International, where he later became a managing director.\nAcross their careers, the team has worked with nearly 200 banks. But this time, they’re going for quality over quantity. “We’re not trying to replicate that scale,” Houston said. “We’re targeting 30 to 40 high-quality bank relationships.”\nWhile the company only began operations last year, the Dubai-based fintech has already secured over 10 million monthly contracted transactions through live bank partnerships across six African markets, Rwanda, Nigeria, Ghana, Egypt, Uganda, and South Africa. \nEnza charges banks on a per-transaction (“per-click”) basis. Those volumes are growing 35 to 40% month-over-month and are expected to double in the next two years.\nThe company bootstrapped in its early years, with the founders funding it themselves. When they decided to raise outside capital, the founders said they didn’t shop the deal widely. \nInstead, Algebra Ventures and Quona Capital led the $6 million seed round. “The Enza leadership team has an impressive track record of starting, growing, and exiting fintech businesses across the continent,” said Tarek Assaad, managing partner at Algebra Ventures, on why his firm backed the two-year-old fintech.\nThe new capital will go toward expanding the team and rolling out new products for its banking clientele across Africa.\n“We founded Enza to solve real infrastructure problems across Africa,” Fekry said. “We’ve spent our careers trying to make sure our families and communities can access financial products as people in Europe or the U.S. at a low cost and anytime they want.”\nTopics\n© 2025 Yahoo.\n'}, {'title': 'DNA testing company 23andMe files for bankruptcy protection, CEO resigns', 'url': 'https://techcrunch.com/2025/03/23/dna-testing-company-23andme-files-for-bankruptcy-protection-ceo-resigns/', 'content': 'Genetics testing company 23andMe has filed for Chapter 11 bankruptcy protection in the U.S. to initiate the sale of its assets. Alongside the announcement, the company’s co-founder and CEO Anne Wojcicki separately said she is leaving the company to become an independent bidder for the company.\n“After a thorough evaluation of strategic alternatives, we have determined that a court-supervised sale process is the best path forward to maximize the value of the business,” Mark Jensen, chair and member of the Special Committee of the Board of Directors, said in a statement.\n“We expect the court-supervised process will advance our efforts to address the operational and financial challenges we face, including further cost reductions and the resolution of legal and leasehold liabilities. We believe in the value of our people and our assets and hope that this process allows our mission of helping people access, understand, and benefit from the human genome to live on for the benefit of customers and patients.”\n“The 23andMe Special Committee released news today indicating their plan to take the company through the Chapter 11 process. While I am disappointed that we have come to this conclusion and my bid was rejected, I am supportive of the company, and I intend to be a bidder. I have resigned as CEO of the company so I can be in the best position to pursue the company as an independent bidder,” Wojcicki said in a post on X.\n23andMe has had a troublesome few years after it went public in 2021. Best known for its saliva-based test kits that offer customers a glimpse into their genetic ancestry, the company has seen its market capitalization plummet more than 99% from a peak of $6 billion after it failed to turn a profit. \nThen in 2023, 23andMe suffered a massive cyberattack in which the data of its nearly 7 million customers, including users’ genetic predisposition and ancestry reports, was stolen by hackers. In September 2024, the company settled a lawsuit related to the data  breach by paying $30 million, and soon afterwards, Wojcicki said she was exploring taking the company private.\nAt the time of writing, the company’s market value was around $48 million, with its stock trading at $1.79.\nRead more:\nTopics\n© 2025 Yahoo.\n'}, {'title': 'Former Cruise CEO Kyle Vogt’s new robotics startup reportedly raises another $150M', 'url': 'https://techcrunch.com/2025/03/23/former-cruise-ceo-kyle-vogts-new-robotics-startup-reportedly-raises-another-150m/', 'content': 'Posted:\nThe Bot Company, the robotics startup founded by former Cruise co-founder and CEO Kyle Vogt, has raised $150 million in a round led by Greenoaks, according to Reuters, citing unnamed sources. Vogt founded the startup with Paril Jain, who led the AI tech team at Tesla, and former Cruise software engineer Luke Holoubek. Vogt did not respond to a request for comment. \nThe fresh round of funding comes less than a year since Vogt launched The Bot Company, which is aiming to produce a robot focused on household chores. The Bot Company launched in May with $150 million from former GitHub CEO and investor Nat Friedman, Pioneer founder and investor Daniel Gross, Spark Capital general partner Nabeel Hyatt, Stripe CEO Patrick Collison, Stripe co-founder John Collison, and Quiet Capital.\nThe new startup was launched about five months after Vogt\xa0resigned as CEO of Cruise, the autonomous vehicle startup that he founded in 2013 and was later acquired by General Motors. His resignation followed an October 2 incident that saw a Cruise vehicle run over and drag a pedestrian 20 feet, after the pedestrian had been hit by a human-driven car.\nTopics\n© 2025 Yahoo.\n'}, {'title': 'Buy now, pay later . . . for a burrito?', 'url': 'https://techcrunch.com/2025/03/23/buy-now-pay-later-for-a-burrito/', 'content': 'Posted:\nIn 2010, a programmer who was mining bitcoin famously made the comically expensive mistake of spending 10,000 bitcoin on two pizzas. As of this writing, those coins would be worth $850 million dollars.\nWhile there are few comparisons to that kind of miscalculation, the prospect of adding interest payments to fast-food orders is raising concerns nonetheless. Stemming from a partnership announced earlier this week between DoorDash and Klarna, customers can now buy a burrito or McDonalds order and pay for it later across four interest-free payments.\nThe deal provides diners — who spend at least $35 —  more flexibility, say both companies. But customers who defer payment on a fast-food delivery are at significantly higher risk of missing one of those interest-free installment payments.\nIndeed, to some, the new partnership is yet another troubling economic sign of the times. Says Chuck Bell of Consumer Reports to the New York Times: “If you don’t pay the bill on time and you start getting multiple late fees, it could end up being a very expensive chile relleno or pad Thai.”\nTopics\n© 2025 Yahoo.\n'}, {'title': 'Barack Obama joins Bluesky', 'url': 'https://techcrunch.com/2025/03/23/barack-obama-joins-bluesky/', 'content': 'Posted:\nSocial network Bluesky, an alternative to X built on open source technologies, has scored a big win in terms of attracting notable users to its platform. The company on Sunday confirmed that former president Barack Obama has joined its service. \nBluesky OOO Rose Wang replied to a post where someone wondered if the account posting as Obama was legitimately him by writing “Confirmed!” \nIn his first few posts on the platform, Obama celebrated the 15th anniversary of the Affordable Care Act (ACA), calling it  “a reminder that change is possible when we fight for progress.”\nThe former president maintains a large following on other social media platforms, including X and Facebook, where he has 130.6 million followers and 55 million followers, respectively. \nHowever, Elon Musk’s X, the platform formerly known as Twitter, has lost a number of users to alternative social networks, including Bluesky, after the 2024 U.S. elections. Musk used X to campaign for President Trump and promote MAGA-aligned political views. He also now holds a prominent position in Trump’s circle thanks to his involvement with DOGE (Department of Government Efficiency), which has been working to reduce the federal government workforce, cut spending, and dismantle government agencies. \nNot surprisingly, many left-leaning U.S. users no longer want to participate on X. Their exodus from the platform led to a large bump in user growth for Bluesky in late 2024. Today, the new social network has over 33 million users and is continuing to grow. \nOther U.S. politicians are already active on Bluesky, including well-known names like Bernie Sanders, Elizabeth Warren, Al Green, Gavin Newsom, and Alexandria Ocasio-Cortez, who became the first person on Bluesky to hit a million followers. Many other representatives, senators, and governors have also joined the network.\n\n\nTopics\n© 2025 Yahoo.\n'}, {'title': 'Browser Use, the tool making it easier for AI ‘agents’ to navigate websites, raises $17M', 'url': 'https://techcrunch.com/2025/03/23/browser-use-the-tool-making-it-easier-for-ai-agents-to-navigate-websites-raises-17m/', 'content': 'We may not have an agreed-upon definition of AI “agent” yet, but a multitude of startups want to create “agentic” tools to automate various tasks online. One such firm, Browser Use, has attracted a ton of interest from developers and investors thanks to its solution that makes websites more “readable” for AI agents.\nBrowser Use told TechCrunch that it has raised a sizable $17 million seed funding round led by Felicis’ Astasia Myers with participation from Paul Graham, A Capital, and Nexus Venture Partners. The company’s funding hasn’t previously been reported.\nBrowser Use, part of Y Combinator’s 2025 winter batch, has gained notoriety in recent months. Chinese startup Butterfly Effect’s use of Brower Use in its viral Manus tool drove awareness to new heights.\nMagnus Müller and Gregor Zunic founded Browser Use last year through ETH Zurich’s Student Project House accelerator. Müller had been working on web-scraping tools for years, and met Zunic in 2024 while the pair were getting their master’s degrees in data science. Together, according to Müller, they came up with the idea of combining web scraping with data science to prompt a browser to perform a task.\nMüller and Zunic built a Browser Use demo in five weeks — and it took off. Subsequently, they open-sourced it. \nBrowser Use essentially breaks down the buttons and elements of a website into a more digestible, “text-like” format for agents. This helps the agents understand the different options and make decisions autonomously.\n“A lot of agents rely on vision-based systems and try and navigate websites through screenshots, and in [the] process, things break,” Müller said. “We convert [websites] into something agents can understand. This approach means we can run the same tasks again and again at a cheaper cost.”\nThere’s an increasing number of AI companies that want to make their agents interact with websites more gracefully, and Müller thinks Browser Use can become a “fundamental layer” serving this need. He added that more than 20 companies in the current Y Combinator Winter batch used Browser Use for their own requirements.\n“There are companies coming to us and saying, ‘What can we do to make it easier for agents to navigate our website?’” Müller said. “There are sites — for example, LinkedIn — that change the way the website works all the time, so agents often fail on sites like those.”\nAccording to Myers, Felicis has been actively looking at the AI agents space for the past several years, and Browser Use felt like the right opportunity to grow the firm’s portfolio there. She said that the company’s founding team — and its open-source-first approach — sealed the deal\n“We think web AI agents are the next frontier that really helps with the end-to-end automation of human tasks,” Myers told TechCrunch. “[W]eb AI agents are this dynamic bridge between static pre-trained models that are mostly text-focused in the ever-changing digital landscape.”\nTopics\n© 2025 Yahoo.\n'}, {'title': 'Jonah Peretti helped shaped digital media — can he do it again?', 'url': 'https://techcrunch.com/2025/03/22/jonah-peretti-helped-shaped-digital-media-can-he-do-it-again/', 'content': 'Jonah Peretti’s career to date has been defined by constant reinvention. As the founder and CEO of BuzzFeed, Peretti has been at the forefront of digital media for almost two decades, navigating changes as the once small startup transformed into, at one point, a multimedia powerhouse. \nBut as the company grows older, one question lingers: Has Peretti’s relentless drive for innovation made him too entrepreneurial to lead BuzzFeed to long-term stability?\n“The nice thing about this field,” Peretti muses, “is you do something [different] all the time. It’s not been 20 years of one job. It’s been 20 years of running a startup, then managing a hyperscaling business, then figuring out how to get the company public . . .and now the explosion in generative AI. There’s always something to keep the job fresh and new.”\nPeretti’s ability to adapt and stay ahead of the curve has been a hallmark of BuzzFeed’s success. Years ago, BuzzFeed enjoyed almost singular success in its uncanny ability to create shareable media. (One video segment, centered on why Disney princes would make “terrible boyfriends,” has racked up 77 million views over the years.)\nToday, BuzzFeed’s focus is on AI, including AI quizzes and other AI-generated content. It’s also pouring some of its resources into a product in development called BF Island, a social network that aims to merge AI with content creation to “spread joy and enable playful creative expression.” So he told Axios last month.\nPeretti routinely discusses the company’s upcoming ventures with the media, even when they’re still in their infancy. Asked why, he tells TechCrunch: “The idea of talking about things you’re working on is that you make the product better . . .When you talk about them, it’s not just with your team but with outside people who might want to collaborate. When there’s a new computing platform, there’s an explosion of creativity. Those are the moments when you can build something new.”\nThis entrepreneurial drive has fueled much of BuzzFeed’s most iconic projects. The company pioneered viral quizzes and listicles, then dove into more serious journalism with BuzzFeed News. But not all of these shifts have paid off. Notably, despite its early success, BuzzFeed News was shuttered in 2023, leaving behind questions about whether the company should have stuck to one clear vision rather than jumping between them.\nBuzzFeed’s shareholders might be pondering the same after the roller-coaster ride they’ve been on. BuzzFeed went public through a special purpose vehicle in 2021, and as part of the transaction, it acquired a media company, Complex Networks, for $300 million in cash and stock. Today, BuzzFeed still trades publicly, but it doesn’t own that business; it sold it for roughly $108 million last year. More recently, BuzzFeed sold another asset from that Complex Networks deal – First We Feast – in a separate $82.5 million all-cash deal. (Explains Peretti of both sales, “They’re not so much tech businesses. They’re more production and talent heavy.”)\nBuzzFeed’s stock, priced at $10 initially, now trades $2.20 per share.\nThis is where the tension lies. Peretti’s embrace of constant reinvention is striking, but it also means that the company’s strategy has often felt like a series of experiments rather than a cohesive long-term plan. The experiments also come at a cost. Peretti says BF Island is a $10 million gamble for now that is not expected to bring in any revenue this year. He adds that BuzzFeed’s “core business” is profitable.\nIt’s a fair question whether BuzzFeed would be better served by a singular, consistent vision for the future – like The New York Times has managed to achieve – rather than betting on one new idea after another. \nUnfortunately, it’s hard to know the answer. The media industry has long favored stability, but Peretti isn’t wrong to believe the current wave of generative AI is revolutionizing how people create and share content. Also, as excited as he sounds about the possibilities, Peretti sounds clear-eyed about the risks.\n“The big thing is that BuzzFeed has given us a lot of experience with new formats,” he says. “We’ve gotten a lot of inbound from people who have ideas and want to collaborate with us.” \nBesides, adds Peretti, who by now is accustomed to balancing growth with financial pressures: “I think the deeper next-level understanding is that just doing the same thing over and over again and trying to to grind it out is actually a more risky strategy than innovating, experimenting, trying new things, keeping an open mind, and trying to figure out new approaches or new ways to win in a really tough market.”\nYou can hear much more from our interview with Peretti in an upcoming episode of StrictlyVC Download; new episodes drop every Tuesday.\nTopics\n© 2025 Yahoo.\n'}]

In [ ]:
import string

def remove_punctuation(input_str):
    """Removes tokens that consist entirely of punctuation and returns the cleaned string."""
    words = input_str.split()  # Split the string into words
    cleaned_words = [word.strip(string.punctuation) for word in words if not all(char in string.punctuation for char in word)]
    return " ".join(cleaned_words)  # Join the cleaned words back into a string

for article in articles:
  print(article['title'])
  print(article['url'])
  input_str = article['content']
  # Call the function to remove punctuation tokens
  cleaned_str = remove_punctuation(input_str)

  # Print the cleaned string
  print(cleaned_str)
  print()


In [ ]:
def text_lowercase(texts):
    return texts.lower()

for article in articles:
  print(article['title'])
  print(article['url'])
  input_str = article['content']
  # Call the function to lower case our article
  cleaned_str = text_lowercase(input_str)

  # Print the cleaned string
  print(cleaned_str)
  print()

In [ ]:
from nltk.corpus import stopwords
def remove_stopwords(text):
    """Removes stopwords from the given string and returns the cleaned string."""
    stop_words = set(stopwords.words("english"))
    stop_words.update(["n't", "'s"])  # Adding specific contractions

    words = text.split()  # Tokenize the input string
    cleaned_words = [word for word in words if word.lower() not in stop_words]

    return " ".join(cleaned_words)  # Return the cleaned sentence as a string

for article in articles:
  print(article['title'])
  print(article['url'])
  input_str = article['content']
  # Call the function to remove_stopwords
  cleaned_text = remove_stopwords(input_str)

  # Print the cleaned string
  print(cleaned_str)
  print()



In [ ]:
def extract_entities(model,text):
    doc = model(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]  # Extract named entities
    return entities


In [ ]:
def extract_relations(model,text):
    doc = model(text)
    relations = []
    for token in doc:
        if token.dep_ in ("nsubj", "nsubjpass") and token.head.dep_ == "ROOT":
            subject = token.text
            predicate = token.head.text
            for child in token.head.children:
                if child.dep_ in ("prep", "agent"):
                    for obj in child.children:
                        if obj.dep_ == "pobj":
                            relations.append((subject, predicate, obj.text))
    return relations

In [ ]:
import rdflib
from rdflib import Graph, URIRef, Literal, Namespace, RDF
# Function to build RDF triples and create a knowledge graph
def build_knowledge_graph(entities, relations):
    g = Graph()
    # Define a namespace
    ex = Namespace("http://example.org/")

    for ent_text, ent_type in entities:
        g.add((URIRef(EX[ent_text.replace(" ", "_")]), RDF.type, URIRef(EX[ent_type])))

    for subj, pred, obj in relations:
        g.add((URIRef(EX[subj.replace(" ", "_")]), URIRef(EX[pred.replace(" ", "_")]), URIRef(EX[obj.replace(" ", "_")])))

    return g

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from pykeen.triples import TriplesFactory
import numpy as np

def pipeline(model,text):
    doc=remove_punctuation(text)
    doc=text_lowercase(doc)
    doc=remove_stopwords(doc)
    print(f"Clean text : {doc}")

    entities = extract_entities(model,doc)
    print(f"\nNamed Entities: {entities}")

    relations = extract_relations(model,text)
    print(f"\nExtracted Relations: {relations}")

    knowledge_graph = build_knowledge_graph(entities, relations)

    # Step 5: Serialize and save the graph
    rdf_output = knowledge_graph.serialize(format="turtle")
    print("\nRDF Knowledge Graph:\n", rdf_output)

    triples = np.array([(str(s), str(p), str(o)) for s, p, o in knowledge_graph], dtype=str)
    tf = TriplesFactory.from_labeled_triples(triples)
    print (tf)

In [ ]:

for article in articles:
  print(article['title'])
  print(article['url'])
  input_str = article['content']
  pipeline(nlp,input_str)
  print()

# PART 2

In [ ]:
! pip install pykeen torch

In [ ]:
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
import numpy as np



triples = np.array([(str(s), str(p), str(o)) for s, p, o in g], dtype=str)
tf = TriplesFactory.from_labeled_triples(triples)

print (tf)

# Split into training and testing sets
train_tf, test_tf = tf.split([0.9, 0.1])  # Adjust the ratio


# Train a TransE model
results = pipeline(
    training=train_tf,
    testing=test_tf,  # Provide test set to avoid error
    model="TransE",
    model_kwargs={"embedding_dim": 50},
    training_kwargs={"num_epochs": 100, "batch_size": 128},
    random_seed=42,
)

# Extract embeddings (handles PyKEEN version differences)
try:
    entity_embeddings = results.model.entity_representations[0]().detach().numpy()
    relation_embeddings = results.model.relation_representations[0]().detach().numpy()
except AttributeError:
    entity_embeddings = results.model.entity_embeddings.weight.detach().numpy()
    relation_embeddings = results.model.relation_embeddings.weight.detach().numpy()

# Print the shape of embeddings
print(f"Entity embeddings shape: {entity_embeddings.shape}")
print(f"Relation embeddings shape: {relation_embeddings.shape}")


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from pykeen.evaluation import RankBasedEvaluator

# Function to find similar entities based on cosine similarity
def find_similar_entities(entity_id, entity_embeddings, top_k=5):

    entity_vector = entity_embeddings[entity_id].reshape(1, -1)
    similarities = cosine_similarity(entity_vector, entity_embeddings)
    most_similar = np.argsort(similarities[0])[-top_k-1:-1][::-1]
    return most_similar

# Example: Use the function to get similar entities
entity_embeddings = results.model.entity_representations[0]().detach().numpy()  # Access entity embeddings
entity_id = 0  # Example entity ID to find similarities
similar_entities = find_similar_entities(entity_id, entity_embeddings, top_k=5)
print(f"Most similar entities to entity {entity_id}: {similar_entities}")

# Link prediction evaluation
# Assuming you have the test set for evaluation
evaluator = RankBasedEvaluator()
evaluation_results = evaluator.evaluate(
    model=results.model,
    mapped_triples=tf.mapped_triples  # Test data
)

# Print the evaluation results (e.g., Mean Rank, Mean Reciprocal Rank, Hit@K)
print("\nLink prediction evaluation results:")
print(evaluation_results)


In [ ]:
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
import numpy as np

# Define RDF triples (subject, predicate, object) - from your pipeline
g = pipeline_graph(nlp, test)
# Convert RDF triples to the PyKEEN format
triples = np.array([(str(s), str(p), str(o)) for s, p, o in g], dtype=str)
tf = TriplesFactory.from_labeled_triples(triples)
# Split the triples into training and testing sets (80% train, 20% test)
train_tf, test_tf = tf.split([0.8, 0.2])

# Train a TransE model with the updated parameters
results = pipeline(
    training=train_tf,
    testing=test_tf,  # Provide the test set to evaluate the model
    model="TransE",
    model_kwargs={"embedding_dim": 20},  # Smaller embedding dimensions
    training_kwargs={
        "num_epochs": 100,
        "batch_size": 128,

    },
    stopper_kwargs={"patience": 5},  # Early stopping with patience of 5 epochs
    random_seed=42,
)

# Extract embeddings (handles PyKEEN version differences)
try:
    entity_embeddings = results.model.entity_representations[0]().detach().numpy()
    relation_embeddings = results.model.relation_representations[0]().detach().numpy()
except AttributeError:
    entity_embeddings = results.model.entity_embeddings.weight.detach().numpy()
    relation_embeddings = results.model.relation_embeddings.weight.detach().numpy()

# Print the shape of the embeddings to verify dimensions
print(f"Entity embeddings shape: {entity_embeddings.shape}")
print(f"Relation embeddings shape: {relation_embeddings.shape}")


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
import numpy as np

def enrich_with_dbpedia(entity_name):
    """
    Retrieve related entities and relations from DBpedia for the given entity.
    """
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    query = """
    SELECT DISTINCT ?related ?relation WHERE {
    ?s rdfs:label "%s"@en .
    ?s ?relation ?related .
    ?related rdfs:label ?label .
    FILTER(LANG(?label) = 'en')
    } LIMIT 10
    """ % entity_name

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"]

def augment_triples_with_dbpedia(triples):
    """
    Augments a list of triples with related entities and relations from DBpedia.
    """
    augmented_triples = []
    for s, p, o in triples:
        # Enrich subject (s) with related entities and relations
        enriched_subjects = enrich_with_dbpedia(s)
        for enriched_subject in enriched_subjects:
            augmented_triples.append((enriched_subject['related']['value'], enriched_subject['relation']['value'], o))

        # Enrich object (o) with related entities and relations
        enriched_objects = enrich_with_dbpedia(o)
        for enriched_object in enriched_objects:
            augmented_triples.append((s, p, enriched_object['related']['value']))

        augmented_triples.append((s, p, o))  # Keep the original triple

    return augmented_triples
g = pipeline_graph(nlp, test)
# Convert RDF triples to the PyKEEN format
triples = np.array([(str(s), str(p), str(o)) for s, p, o in g], dtype=str)
# Augment the triples using DBpedia
augmented_triples = augment_triples_with_dbpedia(triples)


augmented_triples = np.array(augmented_triples, dtype=str)
tf = TriplesFactory.from_labeled_triples(augmented_triples)

print (tf)



In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
import numpy as np


def enrich_with_dbpedia(entity_name):
    """
    Retrieve related entities and relations from DBpedia for the given entity.
    """
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    query = """
    SELECT DISTINCT ?related ?relation WHERE {
      ?s rdfs:label "%s"@en .
      ?s ?relation ?related .
      ?related rdfs:label ?label .
      FILTER(LANG(?label) = 'en')
      FILTER(?relation IN (
        <http://dbpedia.org/ontology/birthPlace>,
        <http://dbpedia.org/ontology/occupation>,
        <http://dbpedia.org/ontology/spouse>,
        <http://dbpedia.org/ontology/child>,
        <http://dbpedia.org/ontology/abstract>
      ))
    } LIMIT 10
    """ % entity_name

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    print(f"DBpedia results for {entity_name}: {results}")  # Debugging print
    return results["results"]["bindings"]

def augment_triples_with_dbpedia(triples):
    """
    Augments a list of triples with related entities and relations from DBpedia.
    """
    augmented_triples = []
    seen_entities = set()
    relations_added = set()  # Track new relations added

    for s, p, o in triples:
        # Enrich subject (s) with related entities and relations
        enriched_subjects = enrich_with_dbpedia(s)
        if enriched_subjects:
            print(f"Found relations for subject {s}")
        for enriched_subject in enriched_subjects:
            enriched_subject_entity = enriched_subject['related']['value']
            enriched_relation = enriched_subject['relation']['value']

            # Only add relations that are meaningful for your knowledge graph
            if enriched_relation in {
                "http://dbpedia.org/ontology/birthPlace",
                "http://dbpedia.org/ontology/occupation",
                "http://dbpedia.org/ontology/spouse",
                "http://dbpedia.org/ontology/child"
            }:
                augmented_triples.append((enriched_subject_entity, enriched_relation, o))
                relations_added.add(enriched_relation)
                seen_entities.add(enriched_subject_entity)

        # Enrich object (o) with related entities and relations
        enriched_objects = enrich_with_dbpedia(o)
        if enriched_objects:
            print(f"Found relations for object {o}")
        for enriched_object in enriched_objects:
            enriched_object_entity = enriched_object['related']['value']
            enriched_relation = enriched_object['relation']['value']

            # Only add relations that are meaningful for your knowledge graph
            if enriched_relation in {
                "http://dbpedia.org/ontology/birthPlace",
                "http://dbpedia.org/ontology/occupation",
                "http://dbpedia.org/ontology/spouse",
                "http://dbpedia.org/ontology/child"
            }:
                augmented_triples.append((s, p, enriched_object_entity))
                relations_added.add(enriched_relation)
                seen_entities.add(enriched_object_entity)

        augmented_triples.append((s, p, o))  # Keep the original triple

    print(f"Total relations added: {len(relations_added)}")
    return augmented_triples


g = pipeline_graph(nlp, test)
# Convert RDF triples to the PyKEEN format
triples = np.array([(str(s), str(p), str(o)) for s, p, o in g], dtype=str)

# Augment the triples using DBpedia
augmented_triples = augment_triples_with_dbpedia(triples)

# Convert augmented triples into PyKEEN format
augmented_triples = np.array(augmented_triples, dtype=str)
tf = TriplesFactory.from_labeled_triples(augmented_triples)



In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
def enrich_with_dbpedia(entity_name):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    query = """
    SELECT DISTINCT ?related ?relation WHERE {
    ?s rdfs:label "%s"@en .
    ?s ?relation ?related .
    ?related rdfs:label ?label .
    FILTER(LANG(?label) = 'en')
    } LIMIT 10
    """ % entity_name

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"]


In [ ]:
print(enrich_with_dbpedia("Putin"))

In [ ]:
from pykeen.triples import TriplesFactory
# Split your triples into training (80%), validation (10%), and test (10%) sets
training, validation, testing = TriplesFactory.from_labeled_triples(triples).split([0.8, 0.1, 0.1])
print(f"Training triples: {training.num_triples}")
print(f"Validation triples: {validation.num_triples}")
print(f"Testing triples: {testing.num_triples}")

In [ ]:
from pykeen.pipeline import pipeline
# Dictionary to store results
model_results = {}
# Test different models
models = ['TransE', 'DistMult', 'ComplEx']
for model_name in models:
 results = pipeline(
 training=training,
 validation=validation,
 testing=testing,
 model=model_name,
 epochs=100,
 embedding_dim=50,
 training_kwargs=dict(batch_size=32),
 random_seed=42,
 )
 model_results[model_name] = results

In [ ]:
def evaluate_model(results, model_name):
 metrics = results.metric_results.to_dict()
 print(f"\nResults for {model_name}:")
 print(f"Mean Rank: {metrics['both']['mean_rank']:.2f}")
 print(f"Mean Reciprocal Rank: {metrics['both']['mean_reciprocal_rank']:.4f}")
 print(f"Hits@1: {metrics['both']['hits_at_1']:.4f}")
 print(f"Hits@3: {metrics['both']['hits_at_3']:.4f}")
 print(f"Hits@10: {metrics['both']['hits_at_10']:.4f}")
for model_name, results in model_results.items():
 evaluate_model(results, model_name)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
def analyze_entity_neighborhood(model, entity_id, k=5):
  # Get entity embeddings
 entity_embeddings = model.entity_embeddings.weight.detach().numpy()
 entity_labels = model.triples_factory.entity_labels

 # Calculate similarities
 similarities = cosine_similarity([entity_embeddings[entity_id]],entity_embeddings)[0]
 most_similar = np.argsort(similarities)[-k-1:-1][::-1]

 print(f"\nMost similar entities to {entity_labels[entity_id]}:")
 for idx in most_similar:
  print(f"{entity_labels[idx]}: {similarities[idx]:.4f}")

In [ ]:
def predict_tail_entities(model, head, relation, k=5):
 # Get predictions
 predictions = model.predict_scores(
 heads=torch.tensor([head]),
 relations=torch.tensor([relation]),
 )

 # Get top k predictions
 top_tails = torch.topk(predictions, k=k, dim=1)

 return [(model.triples_factory.entity_labels[idx.item()],
 score.item())
 for idx, score in zip(top_tails.indices[0],top_tails.values[0])]

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
def visualize_embeddings(model, entity_types=None):
  # Get embeddings and reduce dimensionality
  embeddings = model.entity_embeddings.weight.detach().numpy()
  tsne = TSNE(n_components=2, random_state=42)
  reduced_embeddings = tsne.fit_transform(embeddings)

  # Plot
  plt.figure(figsize=(10, 10))
  if entity_types is None:
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1],alpha=0.5)
  else:
  # Color by entity type
    for entity_type, indices in entity_types.items():
      plt.scatter(reduced_embeddings[indices, 0],reduced_embeddings[indices, 1],label=entity_type, alpha=0.5)
    plt.legend()

  plt.title("Entity Embeddings Visualization")
  plt.show()

In [ ]:
def create_comparison_table(model_results):
 metrics = ['mean_rank', 'mean_reciprocal_rank', 'hits_at_10']
 comparison = {model: {} for model in model_results.keys()}

 for model_name, results in model_results.items():
 for metric in metrics:
 comparison[model_name][metric] =
results.metric_results.to_dict()['both'][metric]

 return pd.DataFrame(comparison).round(4)